import Library


In [1]:
import cv2 as cv
import cvzone
import os
import pickle
import face_recognition as fr
import numpy as np

OPEN WEBCAM


In [2]:
cap = cv.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# imgBackground = cv.imread("Resources/back/2.jpg")
imgBackground = cv.imread("Resources/background.png")

# Importing the mode images into a list

folderModePath = "Resources/Modes"
modePathList = os.listdir(folderModePath)
imgModeList = []

for path in modePathList:
    # توضیح متد https://blog.faradars.org/python-file-system-methods-you-should-know/
    imgModeList.append(cv.imread(os.path.join(folderModePath, path)))

# print(len(imgModeList))
# print(imgModeList)


print("LOADING ENCODE FILE START ...")
# Load the encoding file
file = open("EncodeFile.p", "rb")
encodeListKnownWithIds = pickle.load(file)
file.close()
encodeListKnown, personIds = encodeListKnownWithIds
print(personIds)
print("ENCODE FILE LOADED")

while True:
    success, img = cap.read()

    # koochik kardan tasvir
    imgS = cv.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv.cvtColor(imgS, cv.COLOR_BGR2RGB)

    # mokhtasat chehre ro dar miare
    faceCurFrame = fr.face_locations(imgS)
    # on mokhtasat ro toy tasvir smal encode mikone
    encodeCurFrame = fr.face_encodings(imgS, faceCurFrame)

    imgBackground[162:162+480, 55:55+640] = img
    imgBackground[44:44+633, 808:808+414] = imgModeList[1]

    for encoFace, faceLoc in zip(encodeCurFrame, faceCurFrame):
        matches = fr.compare_faces(encodeListKnown, encoFace)
        faceDis = fr.face_distance(encodeListKnown, encoFace)
        # print("matches : ", matches)
        # print("faceDis : ", faceDis)

        matchIndex = np.argmin(faceDis)
        # print("matchIndex : ", matchIndex)

        if matches[matchIndex]:
            # print("Known Dace Detected")
            # print(personIds[matchIndex])
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
            bbox = 55+x1, 162+y1, x2-x1, y2-y1
            imgBackground = cvzone.cornerRect(imgBackground, bbox, rt=0)

    if success:
        height, width, _ = img.shape
        if height > 0 and width > 0:
            cv.imshow("Face Attendance", imgBackground)
            # cv.imshow("Webcam", img)
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("Invalid image size.")
            break
    else:
        print("Failed to capture frame.")
        break
# print(img.shape)
cv.destroyAllWindows()

LOADING ENCODE FILE START ...
['0', '1', '2', '3', '4', '5', '6', '7', '8']
ENCODE FILE LOADED
